In [600]:
#from __future__ import print_function, division
import os
import zipfile
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

#import cv2 as cv

### Dataset и Dataloader

In [202]:
%pip install git+https://github.com/hassony2/torch_videovision
%pip install av
%pip install torch==1.8.0

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/hassony2/torch_videovision to /tmp/pip-req-build-_347gad5
  Running command git clone -q https://github.com/hassony2/torch_videovision /tmp/pip-req-build-_347gad5
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 735.5 MB 6.6 kB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.7.0 requires torch==1.6.0, but you have torch 1.8.0 which is incompatible.
mmdet 2.3.0rc0+c6b5ca2 requires Pillow<=6.2.2, but you have pillow 8.2.0 which is incompatible.
mmdet 2.3.0rc0+c6b5ca2 requires torch==

In [601]:
from dataloader import LipReadingVideoDataset, LipReadingVideoDatasetZIP, collate_func

In [602]:
#data_path = os.path.normpath('F:/lip_reading_data/LRS3/lrs3_trainval')
data_path_zip = os.path.normpath('data/lrs3_trainval.zip')

In [ ]:
#ds = LipReadingVideoDataset(data_path)

In [603]:
ds_zip = LipReadingVideoDatasetZIP(data_path_zip)

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:809: UserWarning: The following variables cannot be serialized: ds_zip
  warnings.warn(message)


In [283]:
sample = ds_zip[1]
vframes, text, txt_path, video_path = sample.values()

/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:809: UserWarning: The following variables cannot be serialized: ds_zip
  warnings.warn(message)


In [605]:
dl = torch.utils.data.DataLoader(ds_zip, batch_size=4, shuffle=True, collate_fn=collate_func)

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:809: UserWarning: The following variables cannot be serialized: dl, ds_zip
  warnings.warn(message)


In [606]:
for i, (vframes_batch, list_of_sentences) in enumerate(dl):
    print(vframes_batch.shape)
    if i >= 0:
        break

torch.Size([4, 3, 128, 3, 224, 224])


/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives w

### Показ видео

In [ ]:
def video_show(sample):
    vframes = sample['vframes']
    face_df = sample['face']
    video_path = sample['video_path']
    
    cap = cv.VideoCapture(video_path)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            cv.imshow('frame', frame)
            # & 0xFF is required for a 64-bit system
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv.destroyAllWindows()

In [ ]:
video_show(sample)

### Подготовка словаря

In [ ]:
sentences = []

for dir_path, dir_names, file_names in os.walk(data_path):
    # перебрать файлы
    for file_name in file_names:
        if file_name.endswith('.txt'):
            txt_path = os.path.join(dir_path, file_name)
            txt_file = open(txt_path, 'r')
            
            _, text = txt_file.readline().split(':') # прочитанный текст
            text = text.strip()
            
            sentences.append(text)
            txt_file.close()

In [540]:
file = open('sentences.txt', 'r')
sentences = list(map(lambda line: line.strip(), file.readlines()))
file.close()

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:809: UserWarning: The following variables cannot be serialized: file
  warnings.warn(message)


In [541]:
se = sentences[:5]
se

['ONE DAY A YOUNG BOY COMES UPON THE SUNFLOWER WHILE VISITING THE GARDEN AND HE NOTICES HOW WEAK IT LOOKS',
 'JUST LIKE REACHING OUT TO THE SUNFLOWER BY PROVIDING SOMEONE WHO IS NEGLECTED ISOLATED OR FORGOTTEN',
 'THEY ARE INCREDIBLE YET SO OFTEN THEY ARE FORGOTTEN LACKING THE LOVE AND APPRECIATION THEY DESERVE',
 'I WILL HAVE HOPEFULLY MET MY GOAL',
 'THAT WAS COMPLETELY WRONG']

In [542]:
from collections import Counter

token_counts = Counter(tokenizer.tokenize(' '.join(sentences)))

In [543]:
min_count = 1

# tokens from token_counts keys that had at least min_count occurrences throughout the dataset
tokens = []
for key, values in token_counts.items():
    if values >= min_count:
        tokens.append(key)

# Add a special tokens for unknown and empty words
BOS, EOS, UNK, PAD = '_BOS_', '_EOS_', '_UNK_', '_PAD_'
tokens = [PAD, BOS, EOS, UNK] + tokens

In [544]:
len(tokens)

17193

In [545]:
# словари
token_to_id = {tok: i for i, tok in enumerate(tokens)}
id_to_token = {i: tok for tok, i in token_to_id.items()}

### Network

In [607]:
vframes_batch.shape

torch.Size([4, 3, 128, 3, 224, 224])

In [608]:
list_of_sentences

['AND CHILDREN ARE USED TO THAT',
 'ALL THE VOCABULARY',
 'I LOVE SPOKEN WORD',
 'AND PEOPLE WHO ARE EMOTIONAL ABOUT SOMETHING WILL THEREFORE GO OVER THE TOP']

In [548]:
import nltk
tokenizer = nltk.tokenize.WordPunctTokenizer()
#tokenizer = nltk.tokenize.TreebankWordTokenizer()

In [549]:
def sentences_to_tokens(list_of_sentences, tokenaizer):
    list_of_tokens = []
    for sentence in list_of_sentences:
        list_of_tokens.append(tokenizer.tokenize(sentence))
    return list_of_tokens

In [609]:
list_of_tokens = sentences_to_tokens(list_of_sentences, tokenizer)

In [610]:
list_of_tokens

[['AND', 'CHILDREN', 'ARE', 'USED', 'TO', 'THAT'],
 ['ALL', 'THE', 'VOCABULARY'],
 ['I', 'LOVE', 'SPOKEN', 'WORD'],
 ['AND',
  'PEOPLE',
  'WHO',
  'ARE',
  'EMOTIONAL',
  'ABOUT',
  'SOMETHING',
  'WILL',
  'THEREFORE',
  'GO',
  'OVER',
  'THE',
  'TOP']]

In [649]:
my_network = LipReadinNN(token_to_id)

In [613]:
tensor_of_tokens_idx = my_network.list_of_tokens_to_tensor_of_tokens_idx(list_of_tokens)
tensor_of_tokens_idx

tensor([[   1,   16,  323,   38,  981,   27,   54,    2,    0,    0,    0,    0,
            0,    0,    0],
        [   1,  247,   11, 4496,    2,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0],
        [   1,   47,   44,  821, 1230,    2,    0,    0,    0,    0,    0,    0,
            0,    0,    0],
        [   1,   16,  133,   31,   38, 1771,  109,  192,   48,  930,  106,  735,
           11, 2484,    2]])

In [614]:
list_of_tokens_nn = my_network.tensor_of_tokens_idx_to_list_of_tokens(tensor_of_tokens_idx)
list_of_tokens_nn

[['_BOS_',
  'AND',
  'CHILDREN',
  'ARE',
  'USED',
  'TO',
  'THAT',
  '_EOS_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_'],
 ['_BOS_',
  'ALL',
  'THE',
  'VOCABULARY',
  '_EOS_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_'],
 ['_BOS_',
  'I',
  'LOVE',
  'SPOKEN',
  'WORD',
  '_EOS_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_',
  '_PAD_'],
 ['_BOS_',
  'AND',
  'PEOPLE',
  'WHO',
  'ARE',
  'EMOTIONAL',
  'ABOUT',
  'SOMETHING',
  'WILL',
  'THEREFORE',
  'GO',
  'OVER',
  'THE',
  'TOP',
  '_EOS_']]

In [650]:
my_network.train(False)
with torch.no_grad():
    output = my_network(vframes_batch)#, list_of_tokens)

In [651]:
out_sent = list(map(lambda l: ' '.join(l), output))

In [652]:
out_sent

['CONDEMNING CONDEMNING CONDEMNING CONDEMNING INFECTIONS INFECTIONS INFECTIONS INFECTIONS CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING',
 'CONDEMNING CONDEMNING CONDEMNING CONDEMNING INFECTIONS INFECTIONS INFECTIONS INFECTIONS CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING',
 'CONDEMNING CONDEMNING CONDEMNING CONDEMNING INFECTIONS INFECTIONS INFECTIONS INFECTIONS CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING CONDEMNING'

In [655]:
my_network.loss

tensor(7.8678)